# Goal
See how useful Spark is to explore a large data set.

__*PUDF_base_all_tab.txt*__ is 10 GB and 18 M observations on over 250 features.

In [2]:
from os import path, getcwd
import sys

In [9]:
texas_df = spark.read.format('com.databricks.spark.csv') \
    .options(header='true', inferschema='true', delimiter='\t')\
    .load(path.join(getcwd(), "..", "data", "PUDF_base_all_tab.txt"))

In [10]:
print(texas_df.columns)

['discharge_qtr', 'thcic_id', 'PROVIDER_NAME', 'FAC_TEACHING_IND', 'FAC_PSYCH_IND', 'FAC_REHAB_IND', 'FAC_ACUTE_CARE_IND', 'FAC_SNF_IND', 'FAC_LONG_TERM_AC_IND', 'FAC_OTHER_LTC_IND', 'FAC_PEDS_IND', 'SPEC_UNIT_1', 'SPEC_UNIT_2', 'SPEC_UNIT_3', 'SPEC_UNIT_4', 'SPEC_UNIT_5', 'ENCOUNTER_INDICATOR', 'SEX_CODE', 'TYPE_OF_ADMISSION', 'SOURCE_OF_ADMISSION', 'PAT_STATE', 'PAT_ZIP', 'PAT_COUNTRY', 'county', 'public_health_region', 'ADMIT_WEEKDAY', 'LENGTH_OF_STAY', 'pat_age', 'PAT_STATUS', 'RACE', 'ETHNICITY', 'FIRST_PAYMENT_SRC', 'SECONDARY_PAYMENT_SRC', 'TYPE_OF_BILL', 'private_amount', 'semi_private_amount', 'ward_amount', 'icu_amount', 'ccu_amount', 'other_amount', 'pharm_amount', 'medsurg_amount', 'dme_amount', 'used_dme_amount', 'pt_amount', 'ot_amount', 'speech_amount', 'it_amount', 'blood_amount', 'blood_adm_amount', 'or_amount', 'lith_amount', 'card_amount', 'anes_amount', 'lab_amount', 'rad_amount', 'mri_amount', 'op_amount', 'er_amount', 'ambulance_amount', 'pro_fee_amount', 'organ_a

In [6]:
texas_df.show(5)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------